<a href="https://colab.research.google.com/github/Veera83175/Fmml-lab-2024/blob/main/Copy_of_FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
A.The accuracy of a validation set is influenced by the proportion of the data allocated to it. Here’s how changing the size of the validation set can affect its accuracy:

1. **Increasing the Validation Set Size:**
   - **Pros:** A larger validation set provides a more comprehensive evaluation of the model's performance on unseen data. It can lead to more stable and reliable estimates of model accuracy since it is less susceptible to variance and outliers.
   - **Cons:** If the validation set becomes too large, the training set (which is used to train the model) may become smaller. This can reduce the model's ability to learn effectively, potentially leading to poorer generalization and decreased overall model performance.

2. **Reducing the Validation Set Size:**
   - **Pros:** A smaller validation set allows for a larger training set, which can improve the model's ability to learn from more data, potentially enhancing its performance.
   - **Cons:** A smaller validation set may result in less reliable and more variable accuracy estimates. The performance metrics might be influenced more by outliers or anomalies in the validation data, leading to less stable evaluations.

In summary, increasing the validation set size can provide more reliable performance estimates but may reduce the amount of data available for training. Conversely, reducing the validation set size can improve training data availability but might lead to less stable performance estimates. The ideal proportion often depends on the total dataset size and the specific requirements of the problem.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
A.The size of the train and validation sets can impact how well the validation set predicts accuracy on the test set in the following ways:

1. **Large Validation Set Relative to the Training Set:**
   - **Pros:** A larger validation set provides a more robust estimate of model performance on unseen data, which can lead to more accurate predictions of how the model will perform on the test set. This is because the larger validation set is more likely to capture a diverse range of data patterns, making it a better representation of the test set.
   - **Cons:** If the validation set is too large, the training set may be too small, which can hurt the model's learning process. As a result, the model might not perform as well overall, potentially affecting the generalizability of the performance metrics.

2. **Small Validation Set Relative to the Training Set:**
   - **Pros:** A smaller validation set means more data is available for training, which can improve the model's ability to learn complex patterns. This often results in a model with potentially better performance.
   - **Cons:** A smaller validation set might provide a less stable and less reliable estimate of model performance. It might not capture the full variability of the data, which can lead to a less accurate prediction of how the model will perform on the test set. The performance metrics derived from a small validation set can be more sensitive to random fluctuations and less indicative of true model performance.

**Balancing Act:**
- To effectively predict test set accuracy using the validation set, a balance is needed. The validation set should be large enough to provide a reliable estimate of model performance but not so large that it compromises the amount of data available for training.
- In practice, techniques like cross-validation (e.g., k-fold cross-validation) can be used to mitigate these issues by repeatedly using different subsets of the data for training and validation, providing a more stable estimate of performance metrics.

Overall, a well-chosen validation set size, along with appropriate model training practices, helps in obtaining accurate and generalizable performance estimates for the test set.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
A.A common practice in machine learning is to allocate around **10% to 20%** of the dataset for the validation set. This range is generally a good compromise for balancing the factors of accurate performance estimation and sufficient training data.

Here’s a brief breakdown of this allocation:

- **10% Validation Set:** This is often used when you have a large dataset. A smaller percentage can still provide a stable performance estimate without significantly affecting the amount of data available for training.

- **20% Validation Set:** This is a more conservative approach that provides a more robust estimate of model performance, especially useful when dealing with smaller datasets. It allows for a more reliable validation process, though it reduces the training set size accordingly.

In practice, the exact percentage might vary based on the total dataset size and specific problem requirements. For example:

- **Large Datasets:** If the dataset is very large (e.g., millions of samples), even a smaller percentage like 5% can provide a robust validation set. Here, 10% might be more than sufficient.

- **Small Datasets:** For smaller datasets, you might lean towards the higher end of the range or use cross-validation techniques to make the most out of the available data.

Ultimately, the choice of validation set size depends on the specific context and needs of the project, including dataset size, computational resources, and the need for robust performance estimates.
Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
A.Yes, averaging the validation accuracy across multiple splits generally provides more consistent and reliable results. This approach, often referred to as **cross-validation**, helps mitigate the variability associated with a single validation split by evaluating the model on multiple different subsets of the data. Here’s how it benefits the evaluation process:

1. **Reduces Variability:** By using multiple splits, cross-validation smooths out the influence of any particular subset of data that might be unrepresentative or anomalous. This reduces the risk of overfitting or underfitting based on a single validation set.

2. **Provides a Better Estimate:** Averaging results from multiple validation splits offers a more comprehensive view of the model’s performance across different parts of the dataset. This can lead to a more accurate estimate of how the model will perform on new, unseen data.

3. **Increases Robustness:** Cross-validation helps ensure that the model’s performance is not overly dependent on the specific choice of training and validation data. This robustness is particularly important when dealing with smaller datasets, where a single split might not be fully representative.

Common forms of cross-validation include:

- **k-Fold Cross-Validation:** The dataset is divided into `k` equally sized folds. The model is trained `k` times, each time using a different fold as the validation set and the remaining folds as the training set. The final performance estimate is the average of the `k` validation scores.

- **Leave-One-Out Cross-Validation (LOOCV):** A special case of k-Fold cross-validation where `k` equals the number of data points in the dataset. Each data point is used once as the validation set while the rest serve as the training set.

- **Stratified k-Fold Cross-Validation:** Similar to k-Fold, but it ensures that each fold maintains the same proportion of class labels as the full dataset, which is particularly useful for imbalanced datasets.

In summary, averaging validation accuracy across multiple splits improves the reliability and stability of performance estimates, making it a preferred practice in many machine learning tasks.
2. Does it give more accurate estimate of test accuracy?
A.Averaging validation accuracy across multiple splits, as in cross-validation, provides a more accurate estimate of a model's performance on unseen data compared to using a single validation split. Here’s why:

1. **More Comprehensive Evaluation:** Cross-validation evaluates the model on multiple different subsets of data. This process provides a broader view of how the model performs across various parts of the dataset, leading to a more robust estimate of performance.

2. **Reduced Overfitting Risk:** By averaging results across multiple splits, cross-validation helps to mitigate the risk that the model's performance is overly dependent on a specific validation set. This reduces the likelihood of overfitting to a particular subset of data, leading to a more generalized performance estimate.

3. **Better Performance Metrics:** Cross-validation helps to smooth out the influence of any particular data subset that might be unusually easy or difficult to predict. This results in a more reliable performance metric that better reflects the model's ability to generalize.

4. **Estimate Variability:** Cross-validation provides an estimate of performance variability by showing how results differ across different folds. This variability information is valuable for understanding the stability of the model’s performance.

However, it's important to note that cross-validation estimates are still based on the validation sets and not directly on the test set. The final test accuracy should be assessed using a separate test set that was not involved in the training or validation process to get an unbiased estimate of performance on truly unseen data.

In summary, cross-validation generally gives a more accurate estimate of how the model will perform on unseen data by reducing the impact of a single validation set's idiosyncrasies and providing a more comprehensive view of model performance.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
A.In the context of cross-validation, the number of iterations—typically referred to as the number of folds in k-Fold Cross-Validation—does influence the quality of the performance estimate. Here’s how the number of folds affects the estimate:

1. **Higher Number of Folds (More Iterations):**
   - **More Accurate Estimates:** Increasing the number of folds generally leads to a more accurate estimate of model performance. With more folds, each individual fold contains a smaller fraction of the data, and the model is trained and validated on nearly all of the data. This approach often provides a more precise estimate of how the model will perform on unseen data.
   - **Less Bias:** Higher k reduces bias in the estimate because each data point is used for both training and validation, providing a more representative evaluation.
   - **More Computational Cost:** More folds mean that the model is trained and validated more times, increasing computational expense and time. For large datasets or complex models, this can become significant.

2. **Lower Number of Folds (Fewer Iterations):**
   - **Higher Variability:** Fewer folds mean that each validation set is larger, but the training sets are smaller. This can lead to higher variance in performance estimates, as the model might be evaluated on data that is less representative of the overall dataset.
   - **Less Computational Cost:** Fewer folds reduce the number of model training iterations, which can be advantageous in terms of computational resources and time. However, this comes at the cost of potentially less accurate estimates of performance.

**Special Cases:**
- **Leave-One-Out Cross-Validation (LOOCV):** This is an extreme case where the number of folds equals the number of data points. Each iteration uses one data point as the validation set and the rest as the training set. LOOCV provides an almost unbiased estimate of performance but can be computationally expensive and might suffer from high variance if the dataset is small.

- **Stratified k-Fold Cross-Validation:** For classification tasks, using stratified folds ensures that each fold maintains the proportion of class labels found in the entire dataset, which helps in providing a more reliable estimate, especially for imbalanced datasets.

In summary, increasing the number of iterations (or folds) in cross-validation generally improves the accuracy of the performance estimate by reducing bias and providing a more comprehensive evaluation. However, it also increases computational cost, so a balance must be struck based on dataset size, model complexity, and available computational resources.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
A.
Increasing the number of iterations (or folds) in cross-validation can help mitigate some issues associated with small training or validation datasets, but it doesn't fully solve the problem. Here's how it impacts small datasets:

1. **Small Training Dataset:**
   - **Impact of More Iterations:** Increasing the number of folds in cross-validation means that each iteration uses a larger portion of the data for training. This can be beneficial for small datasets because it allows the model to be trained on almost all available data, thus maximizing the amount of data used for learning in each iteration.
   - **Limitations:** Even with many folds, a very small dataset may still lead to models that have high variance due to limited data. The model might not generalize well if it’s trained on very small subsets. Additionally, with very small datasets, you may encounter high variance in performance estimates due to the small size of each fold.

2. **Small Validation Dataset:**
   - **Impact of More Iterations:** By increasing the number of folds, each validation set in cross-validation will be smaller. This can make each fold's performance estimate less reliable due to the small number of samples used for validation in each iteration.
   - **Limitations:** Even with increased iterations, very small validation sets may not provide a stable estimate of model performance. The validation accuracy might be highly variable and not representative of how the model will perform on new, unseen data.

**Practical Considerations:**
- **Cross-Validation Benefits:** While increasing the number of folds helps to make the most of the available data and provides a more comprehensive performance estimate, it doesn't fully address the fundamental limitations of having too little data. Cross-validation helps to ensure that all data points are used for both training and validation, but it does not compensate for the inherent limitations of very small datasets.
  
- **Alternative Approaches:** For very small datasets, you might consider additional strategies such as:
  - **Data Augmentation:** Generating more training data through techniques like augmentation can help improve model performance.
  - **Transfer Learning:** Using pre-trained models on related tasks and fine-tuning them on your small dataset can be effective.
  - **Regularization:** Employing strong regularization techniques can help prevent overfitting when training on small datasets.

In summary, increasing the number of cross-validation folds can improve the use of a small dataset by allowing more comprehensive training and evaluation. However, it does not replace the need for sufficient data, and other strategies might be necessary to address the challenges posed by very small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.